In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp2d

from numba import njit
from src.model_tech_dice import hjb_post_damage_post_tech, hjb_post_damage_pre_tech
from src.model_tech_dice import hjb_pre_damage_post_tech, hjb_pre_damage_pre_tech
from src.utilities import find_nearest_value
from src.simulation_2d import EvolutionState, simulation_dice_prob

In [2]:
##################
ξ_a = 2./100
ξ_p = 7.5
arrival = 20
τ = 2.
y_bar_lower = 1.5
n_model = 20
##################

I_g_first = 1./arrival
I_g_second = 1./arrival
ξ_g_first = ξ_p
ξ_g_second = ξ_p
ξ_b = ξ_p

# Model parameters
δ = 0.01
α = 0.115
κ = 6.667
μ_k = -0.043
σ_k = np.sqrt(0.0087**2 + 0.0038**2)

theta = 3
lambda_bar = 0.1206
vartheta_bar = 0.0453

γ_1 = 1.7675/10000
γ_2 = .0022*2

γ_3_lower = 0.
γ_3_upper = 1./3

# Compute γ_3 for the n models
def Γ(y, y_bar, γ_1, γ_2, γ_3):
    return γ_1 * y + γ_2 / 2 * y ** 2 + γ_3 / 2 * (y > y_bar) * (y - y_bar) ** 2

prop_damage_lower = np.exp(-Γ(2.5, 2., γ_1, γ_2, γ_3_upper))
prop_damage_upper = np.exp(-Γ(2.5, 2., γ_1, γ_2, γ_3_lower))
γ_3 = (-np.log(np.linspace(prop_damage_lower, prop_damage_upper, n_model)) - γ_1 * 2.5 - γ_2 / 2 * 2.5**2) / .5**2 * 2
γ_3.sort()
γ_3[0] = 0
πd_o = np.ones(n_model)/n_model

θ = pd.read_csv('data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ)/len(θ)
σ_y = 1.2*np.mean(θ)

# Grid setting
k_step = .1
k_grid = np.arange(0, 10+k_step, k_step)

y_step = .1
y_grid_long = np.arange(0., 5.+y_step, y_step)
y_grid_short = np.arange(0., 2.5+y_step, y_step)
n_bar = find_nearest_value(y_grid_long, τ) + 1

# Tech jump
lambda_bar_first = lambda_bar / 2
vartheta_bar_first = vartheta_bar / 2
lambda_bar_second = 1e-9
vartheta_bar_second = 0.

In [3]:
# Solve post damage, post second tech jump models
model_post_damage_post_second_tech = []
for i, γ_3_i in enumerate(γ_3):
    model_args = (δ, α, κ, μ_k, σ_k, θ, πc_o, σ_y, ξ_a, ξ_b, γ_1, γ_2, γ_3_i, τ, theta, lambda_bar_second, vartheta_bar_second)
    if i == 0:
        v_guess = None
    else:
        v_guess = model_res['v']
    model_res = hjb_post_damage_post_tech(k_grid, y_grid_long, model_args, v0=v_guess, ϵ=1., fraction=.05,
                                          tol=1e-6, max_iter=2000, print_iteration=False)
    model_post_damage_post_second_tech.append(model_res)

Converged. Total iteration 1636: LHS Error: 9.93537241811282e-07; RHS Error 9.61302791147034e-07
Converged. Total iteration 5: LHS Error: 9.93164348983555e-07; RHS Error 9.632327593322998e-07
Converged. Total iteration 3: LHS Error: 9.85425117505656e-07; RHS Error 9.564560432551161e-07
Converged. Total iteration 2: LHS Error: 9.790725243874476e-07; RHS Error 9.513476655342856e-07
Converged. Total iteration 1: LHS Error: 9.866284926829394e-07; RHS Error 9.592987728623825e-07
Converged. Total iteration 1: LHS Error: 9.768783684194204e-07; RHS Error 9.48565491043496e-07
Converged. Total iteration 1: LHS Error: 9.669053042671294e-07; RHS Error 9.388116204434027e-07
Converged. Total iteration 1: LHS Error: 9.567093002260663e-07; RHS Error 9.290674958251088e-07
Converged. Total iteration 1: LHS Error: 9.462909247304196e-07; RHS Error 9.19122395970362e-07
Converged. Total iteration 1: LHS Error: 9.368314977109549e-07; RHS Error 9.089777028847834e-07
Converged. Total iteration 1: LHS Error: 9.

In [4]:
# Solve post damage, post first tech jump models
model_post_damage_post_first_tech = []
for i, γ_3_i in enumerate(γ_3):
    model_args = (δ, α, κ, μ_k, σ_k, θ, πc_o, σ_y, ξ_a, ξ_b, ξ_g_second, I_g_second,
                  model_post_damage_post_second_tech[i]['v'], γ_1, γ_2,
                  γ_3_i, τ, theta, lambda_bar_first, vartheta_bar_first)
    if i == 0:
        v_guess = model_res['v']
    else:
        v_guess = model_res['v']
    model_res = hjb_post_damage_pre_tech(k_grid, y_grid_long, model_args, v0=v_guess, ϵ=1., fraction=.05,
                                         tol=1e-6, max_iter=2000, print_iteration=False)
    model_post_damage_post_first_tech.append(model_res)

Converged. Total iteration 428: LHS Error: 9.913700296237948e-07; RHS Error 0.0010481680422103068
Converged. Total iteration 240: LHS Error: 9.804799674384412e-07; RHS Error 0.001636487949731702
Converged. Total iteration 226: LHS Error: 9.602532600183622e-07; RHS Error 0.002229345181532949
Converged. Total iteration 195: LHS Error: 9.974848147464854e-07; RHS Error 0.0026726994207066923
Converged. Total iteration 196: LHS Error: 9.617398788464016e-07; RHS Error 0.0030186137450058936
Converged. Total iteration 196: LHS Error: 9.779326717307413e-07; RHS Error 0.0033028126985463757
Converged. Total iteration 196: LHS Error: 9.901810642531927e-07; RHS Error 0.003543166736896808
Converged. Total iteration 196: LHS Error: 9.996378906862446e-07; RHS Error 0.0037504339432921086
Converged. Total iteration 197: LHS Error: 9.500512589966092e-07; RHS Error 0.003928430582661235
Converged. Total iteration 197: LHS Error: 9.572099486376828e-07; RHS Error 0.0040784019315047715
Converged. Total iterati

In [5]:
# Solve post damage pre tech jump models
model_post_damage_pre_tech = []
for i, γ_3_i in enumerate(γ_3):
    model_args = (δ, α, κ, μ_k, σ_k, θ, πc_o, σ_y, ξ_a, ξ_b, ξ_g_first, I_g_first,
                  model_post_damage_post_first_tech[i]['v'], γ_1, γ_2,
                  γ_3_i, τ, theta, lambda_bar, vartheta_bar)
    if i == 0:
        v_guess = model_res['v']
    else:
        v_guess = model_res['v']
    model_res = hjb_post_damage_pre_tech(k_grid, y_grid_long, model_args, v0=v_guess, ϵ=1., fraction=.05,
                                         tol=1e-6, max_iter=2000, print_iteration=False)
    model_post_damage_pre_tech.append(model_res)

Converged. Total iteration 668: LHS Error: 9.760090051713632e-07; RHS Error 0.00221280271869162
Converged. Total iteration 301: LHS Error: 9.897211157294805e-07; RHS Error 0.003416536634929912
Converged. Total iteration 333: LHS Error: 9.871638440017705e-07; RHS Error 0.004381205852627801
Converged. Total iteration 240: LHS Error: 9.86811642178509e-07; RHS Error 0.005123053290660096
Converged. Total iteration 240: LHS Error: 9.865384527074639e-07; RHS Error 0.005720087022233145
Converged. Total iteration 240: LHS Error: 9.84198777587153e-07; RHS Error 0.006223921674475719
Converged. Total iteration 240: LHS Error: 9.916727776726475e-07; RHS Error 0.00666061722818867
Converged. Total iteration 240: LHS Error: 9.922656545313657e-07; RHS Error 0.007044124600277257
Converged. Total iteration 240: LHS Error: 9.952112804967328e-07; RHS Error 0.0073804857084844855
Converged. Total iteration 240: LHS Error: 9.925604445015779e-07; RHS Error 0.007692004233060601
Converged. Total iteration 240: L

In [6]:
# Solve pre damage, post second tech models
v_i_short = []
for model in model_post_damage_post_second_tech:
    temp = np.zeros((len(k_grid), len(y_grid_short)))
    for i in range(temp.shape[1]):
        temp[:, i] = model['v'][:, n_bar-1]
    v_i_short.append(temp)
v_i_short = np.array(v_i_short)

model_args = (δ, α, κ, μ_k, σ_k, θ, πc_o, σ_y, ξ_a, ξ_b, ξ_p, πd_o, v_i_short, γ_1, γ_2, theta, lambda_bar_second, vartheta_bar_second, y_bar_lower)
model_pre_damage_post_second_tech = hjb_pre_damage_post_tech(k_grid, y_grid_short, model_args=model_args, v0=np.mean(v_i_short, axis=0),
                                                             ϵ=1., fraction=.05, tol=1e-6, max_iter=2_000, print_iteration=True)

Iteration 1: LHS Error: 9.015535624712356e-07; RHS Error 8.740777035498581e-07
Converged. Total iteration 1: LHS Error: 9.015535624712356e-07; RHS Error 8.740777035498581e-07


In [7]:
# Solve pre damage, post first tech models
v_i_short = []
for model in model_post_damage_post_first_tech:
    temp = np.zeros((len(k_grid), len(y_grid_short)))
    for i in range(temp.shape[1]):
        temp[:, i] = model['v'][:, n_bar-1]
    v_i_short.append(temp)
v_i_short = np.array(v_i_short)

model_args = (δ, α, κ, μ_k, σ_k, θ, πc_o, σ_y, ξ_a, ξ_b, ξ_g_second, ξ_p,
              πd_o, v_i_short, I_g_second, model_pre_damage_post_second_tech['v'], γ_1, γ_2,
              theta, lambda_bar_first, vartheta_bar_first, y_bar_lower)

model_pre_damage_post_first_tech = hjb_pre_damage_pre_tech(k_grid, y_grid_short, model_args=model_args, v0=np.mean(v_i_short, axis=0),
                                                           ϵ=.1, fraction=.05, tol=1e-6, max_iter=2_000, print_iteration=False)

Converged. Total iteration 1667: LHS Error: 0.0; RHS Error 0.003279944132593471


In [8]:
# Solve pre damage, pre tech models
v_i_short = []
for model in model_post_damage_pre_tech:
    temp = np.zeros((len(k_grid), len(y_grid_short)))
    for i in range(temp.shape[1]):
        temp[:, i] = model['v'][:, n_bar-1]
    v_i_short.append(temp)
v_i_short = np.array(v_i_short)

model_args = (δ, α, κ, μ_k, σ_k, θ, πc_o, σ_y, ξ_a, ξ_b, ξ_g_first, ξ_p,
              πd_o, v_i_short, I_g_first, model_pre_damage_post_first_tech['v'], γ_1, γ_2,
              theta, lambda_bar, vartheta_bar, y_bar_lower)

model_pre_damage_pre_tech = hjb_pre_damage_pre_tech(k_grid, y_grid_short, model_args=model_args, v0=np.mean(v_i_short, axis=0),
                                                    ϵ=.1, fraction=.05, tol=1e-6, max_iter=2_000, print_iteration=False)

Converged. Total iteration 1823: LHS Error: 0.0; RHS Error 0.010207346131995365


## Simulation


In [9]:
# Case 1) : damage jump intensity & probability (no tech jump, no damage jump)
T_plots = 50
sim_args = (κ, μ_k, σ_k, np.mean(θ))
et, kt, yt, _, gt, πct, ht = simulation_dice_prob(sim_args, k_grid, y_grid_short,
                                      model_pre_damage_pre_tech['e'],
                                      model_pre_damage_pre_tech['i'],
                                      model_pre_damage_pre_tech['g'],
                                      model_pre_damage_pre_tech['h'],
                                      model_pre_damage_pre_tech['πc'],
                                      K0=85/0.115, y0=1.1, T=T_plots)

def damage_intensity(y, y_bar_lower):
    r1 = 1.5
    r2 = 2.5
    return r1 * (np.exp(r2/2 * (y - y_bar_lower)**2) - 1) * (y >= y_bar_lower)

intensity_dmg = damage_intensity(yt, y_bar_lower)
intensity_distortion = np.mean(gt, axis=0)
distorted_damage_probs = gt / np.mean(gt, axis=0) / n_model

In [10]:
np.save('new_intensity_dmg_7p5.npy', intensity_dmg)
np.save(f'new_dmg_intensity_distort_{ξ_p}.npy', intensity_distortion)

In [11]:
# Case 2) : tech jump intensity & probability (no tech jump, no damage jump)
T_plots = 50
sim_args = (κ, μ_k, σ_k, np.mean(θ))
_, kt, yt, _, gt_tech, _, _ = simulation_dice_prob(sim_args, k_grid, y_grid_short,
                                      model_pre_damage_pre_tech['e'],
                                      model_pre_damage_pre_tech['i'],
                                      [model_pre_damage_pre_tech['g_tech']],
                                      model_pre_damage_pre_tech['h'],
                                      model_pre_damage_pre_tech['πc'],
                                      K0=85/0.115, y0=1.1, T=T_plots)

# Case 3) : tech jump intensity (tech jumped once, no damage jump)
sim_args = (κ, μ_k, σ_k, np.mean(θ))
_, kt_new, yt_new, _, gt_tech_new, _, _ = simulation_dice_prob(sim_args, k_grid, y_grid_short,
                                      model_pre_damage_post_first_tech['e'],
                                      model_pre_damage_post_first_tech['i'],
                                      [model_pre_damage_post_first_tech['g_tech']],
                                      model_pre_damage_post_first_tech['h'],
                                      model_pre_damage_post_first_tech['πc'],
                                      K0=np.exp(kt[arrival]), y0=yt[arrival], T=T_plots)

In [12]:
np.save('new_gt_tech_7p5.npy', gt_tech[0])
np.save('new_gt_tech_new_7p5.npy', gt_tech_new[0])